In [76]:
import pandas as pd
import pyodbc
import sqlite3

DB = {'servername': 'XPS15\\SQLEXPRESS', 'database': 'gecombineerde_database2'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')

export_cursor = export_conn.cursor()

In [49]:
#csv bestanden-inventory_levels
inventory_dataframe= pd.read_csv("inventory_levels_train.csv")
inventory_dataframe

,Unnamed: 0,INVENTORY_YEAR,INVENTORY_MONTH,PRODUCT_NUMBER,INVENTORY_COUNT
0,0,2023,4,48,1932
1,1,2023,4,49,1400
2,2,2023,4,50,21705
3,3,2023,4,51,9710
4,4,2023,4,52,5616
...,...,...,...,...,...
3538,3790,2023,4,43,6147
3539,3791,2023,4,44,5822
3540,3792,2023,4,45,1676
3541,3793,2023,4,46,1802


In [50]:
# Verwijder de ongewenste kolom als die bestaat
inventory_dataframe = inventory_dataframe.drop(columns=["Unnamed"], errors="ignore")


# SQL INSERT statement
sql = "INSERT INTO inventory (INVENTORY_YEAR, INVENTORY_MONTH, PRODUCT_NUMBER, INVENTORY_COUNT) VALUES (?, ?, ?, ?)"

# Data omzetten naar lijst van tuples
data_to_insert = inventory_dataframe[['INVENTORY_YEAR', 'INVENTORY_MONTH', 'PRODUCT_NUMBER', 'INVENTORY_COUNT']].values.tolist()

# Meerdere rijen tegelijk invoegen
try:
    export_cursor.executemany(sql, data_to_insert)
    export_conn.commit()
except Exception as e:
    print(" Fout bij executemany:", e)


# Opslaan en connectie sluiten
export_conn.commit()
export_cursor.close()
export_conn.close()
print("Data succesvol ingevoegd!")

Data succesvol ingevoegd!


In [52]:
#CSV bestand weer- product_forecast
product_forcast_dataframe= pd.read_csv("product_forecast_train.csv")
product_forcast_dataframe

,PRODUCT_NUMBER,YEAR,MONTH,EXPECTED_VOLUME
0,45,2023,12,90
1,46,2023,12,84
2,47,2023,12,468
3,48,2023,12,91
4,49,2023,12,35
...,...,...,...,...
3524,37,2022,1,60
3525,38,2022,1,51
3526,39,2022,1,235
3527,40,2022,1,568


In [53]:
# SQL INSERT statement
sql = "INSERT INTO product_forecast (PRODUCT_NUMBER, YEAR, MONTH, EXPECTED_VOLUME) VALUES (?, ?, ?, ?)"

# Data omzetten naar lijst van tuples
data_to_insert = product_forcast_dataframe[['PRODUCT_NUMBER', 'YEAR', 'MONTH', 'EXPECTED_VOLUME']].values.tolist()

# Meerdere rijen tegelijk invoegen
try:
    export_cursor.executemany(sql, data_to_insert)
    export_conn.commit()
except Exception as e:
    print(" Fout bij executemany:", e)


# Opslaan en connectie sluiten
export_conn.commit()
export_cursor.close()
export_conn.close()
print("Data succesvol ingevoegd!")

Data succesvol ingevoegd!


In [55]:
# SQLite-bestanden inladen en data overzetten naar SQL Server
sqlite_files = ['go_crm_train.sqlite', 'go_sales_train.sqlite', 'go_staff_train.sqlite']

# Functie om gegevens naar SQL Server te sturen
def insert_to_sql_server(table, columns, data_to_insert):
    # Bouw de SQL INSERT-statement op basis van de kolomnamen
    sql = f"INSERT INTO {table} ({', '.join(columns)}) VALUES ({', '.join(['?'] * len(columns))})"
    try:
        # Meerdere rijen tegelijk invoegen
        export_cursor.executemany(sql, data_to_insert)
        export_conn.commit()
        print(f"Data voor {table} succesvol ingevoerd!")
    except Exception as e:
        print(f"Fout bij {table}: {e}")
        # Eventueel: export_conn.rollback()

# Loop over elk SQLite-bestand
for sqlite_file in sqlite_files:
    print(f"Verbinding maken met SQLite bestand: {sqlite_file}")

    # Maak verbinding met SQLite
    sqlite_conn = sqlite3.connect(sqlite_file)

    # Haal alle tabellen op uit het SQLite-bestand
    tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql_query(tables_query, sqlite_conn)['name'].tolist()

    # Loop over alle tabellen en voer de inserts uit
    for table in tables:
        print(f"Invoeren van data voor tabel: {table}")

        # Haal de kolommen en data uit de tabel
        query = f"SELECT * FROM {table};"
        dataframe = pd.read_sql_query(query, sqlite_conn)

        # Zet de data om naar een lijst van lijsten
        data_to_insert = dataframe.values.tolist()
        columns = dataframe.columns.tolist()  # De kolomnamen

        # Verstuur de data naar SQL Server
        insert_to_sql_server(table, columns, data_to_insert)

    # Sluit de verbinding met de SQLite database
    sqlite_conn.close()

# Sluit de verbinding met SQL Server
export_cursor.close()
export_conn.close()


Verbinding maken met SQLite bestand: go_crm_train.sqlite
Invoeren van data voor tabel: age_group
Data voor age_group succesvol ingevoerd!
Invoeren van data voor tabel: country
Data voor country succesvol ingevoerd!
Invoeren van data voor tabel: retailer_segment
Data voor retailer_segment succesvol ingevoerd!
Invoeren van data voor tabel: sales_territory
Data voor sales_territory succesvol ingevoerd!
Invoeren van data voor tabel: retailer
Fout bij retailer: ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 5 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLExecDirectW)')
Invoeren van data voor tabel: retailer_site
Data voor retailer_site succesvol ingevoerd!
Invoeren van data voor tabel: retailer_type
Data

In [11]:
#tabellen die niet gelukt zijn:
#country (Probleem met een FOREIGN KEY constraint in de sales_territory tabel)
#retailer (Probleem met een fout in de datastroom bij een float type
#retailer_site (Probleem met een FOREIGN KEY constraint in de country tabel)
#sales_demographic (Probleem met een FOREIGN KEY constraint in de retailer_headquarters tabel)
#retailer_contact (Probleem met een fout in de datastroom bij een float type)
#order_details (Probleem met ongeldige kolomnamen)
#order_header (Probleem met een FOREIGN KEY constraint in de order_method tabel)
#returned_item (Probleem met een FOREIGN KEY constraint in de order_details tabel)
#sales_branch (Probleem met een FOREIGN KEY constraint in de country tabel)
#sales_staff (Probleem met een FOREIGN KEY constraint in de sales_branch tabel)
#retailer_site (Probleem met een FOREIGN KEY constraint in de country tabel)
#satisfaction (Probleem met een FOREIGN KEY constraint in de sales_staff tabel)

#in totaal 12 tabellen zijn mis gegaan.


In [46]:
#datastroom probleem bij retailer tabel
# Lijst van SQLite-bestanden
sqlite_files = ['go_crm_train.sqlite', 'go_sales_train.sqlite', 'go_staff_train.sqlite']

# Functie om te controleren in welk bestand de 'retailer'-tabel zit
def find_table_in_sqlite(table_name, sqlite_files):
    for sqlite_file in sqlite_files:
        conn = sqlite3.connect(sqlite_file)
        cursor = conn.cursor()

        # Controleer of de tabel bestaat in het huidige SQLite-bestand
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?;", (table_name,))
        result = cursor.fetchone()

        if result:
            print(f"Tabel '{table_name}' gevonden in {sqlite_file}")
            conn.close()
            return sqlite_file  # Geef het bestand terug waar de tabel is gevonden

        conn.close()

    print(f"Tabel '{table_name}' niet gevonden in de opgegeven bestanden.")
    return None

# Zoek in welk bestand de 'retailer' tabel zich bevindt
sqlite_file = find_table_in_sqlite("retailer", sqlite_files)

if sqlite_file:
    # Verbinding maken met het juiste SQLite-bestand
    conn = sqlite3.connect(sqlite_file)

    # Haal de kolomnamen en types op
    query = "PRAGMA table_info(retailer);"
    columns_info = pd.read_sql_query(query, conn)

    print("Kolommen en types in 'retailer' tabel:")
    print(columns_info)

    # Haal alle data op uit de 'retailer'-tabel
    query = "SELECT * FROM retailer;"
    retailer_dataframe = pd.read_sql_query(query, conn)

    # Controleer of er problemen zijn met float kolommen
    print("Data preview:")
    retailer_dataframe

    # Sluit de verbinding
    conn.close()
else:
    print("Kan de 'retailer' tabel niet vinden.")

Tabel 'retailer' gevonden in go_crm_train.sqlite
Kolommen en types in 'retailer' tabel:
   cid                name     type  notnull dflt_value  pk
0    0       RETAILER_CODE  INTEGER        0       None   1
1    1     RETAILER_CODEMR  INTEGER        0       None   0
2    2        COMPANY_NAME     TEXT        0       None   0
3    3  RETAILER_TYPE_CODE  INTEGER        1       None   0
Data preview:


In [41]:
#retailer tabel in database zetten
# Verwijder lege kolommen of kolommen zonder naam
retailer_dataframe = retailer_dataframe.drop(columns=[""], errors="ignore")

# SQL INSERT statement
sql = "INSERT INTO retailer (RETAILER_CODE, RETAILER_CODEMR, COMPANY_NAME, RETAILER_TYPE_CODE) VALUES (?, ?, ?, ?)"

# Data omzetten naar lijst van tuples
data_to_insert = retailer_dataframe[['RETAILER_CODE', 'RETAILER_CODEMR', 'COMPANY_NAME', 'RETAILER_TYPE_CODE']].values.tolist()

# Meerdere rijen tegelijk invoegen
try:
    export_cursor.executemany(sql, data_to_insert)
    export_conn.commit()
except Exception as e:
    print(" Fout bij executemany:", e)


# Opslaan en connectie sluiten
export_conn.commit()
export_cursor.close()
export_conn.close()
print("Data succesvol ingevoegd!")

 Fout bij executemany: ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 5 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLExecDirectW)')
Data succesvol ingevoegd!


In [44]:
print(retailer_dataframe.dtypes)  # Bekijk het datatype van elke kolom
print(retailer_dataframe.head())  # Controleer de eerste rijen
print(retailer_dataframe['RETAILER_TYPE_CODE'].unique())  # Bekijk unieke waarden in de kolom



RETAILER_CODE           int64
RETAILER_CODEMR       float64
COMPANY_NAME           object
RETAILER_TYPE_CODE      int64
dtype: object
   RETAILER_CODE  RETAILER_CODEMR COMPANY_NAME  RETAILER_TYPE_CODE
0              1              NaN         None                   5
1             15            234.0         None                   1
2             23            229.0         None                   3
3             32            230.0         None                   4
4             33            228.0         None                   6
[5 1 3 4 6 7 2 8]


In [59]:
import pandas as pd
import sqlite3

sqlite_files = ['go_crm_train.sqlite', 'go_sales_train.sqlite', 'go_staff_train.sqlite']

for sqlite_file in sqlite_files:
    print(f"Verwerken van {sqlite_file}...")
    sqlite_conn = sqlite3.connect(sqlite_file)

    # Haal alle tabellen op
    tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", sqlite_conn)['name'].tolist()

    for table in tables:
        print(f"  Exporteer tabel {table} naar CSV...")
        df = pd.read_sql(f"SELECT * FROM {table}", sqlite_conn)

        # Data Cleaning
        df = df.replace(r'^\s*$', pd.NA, regex=True)  # Vervang lege strings door NaN

        # Hernoem kolommen indien nodig (pas dit aan per tabel)
        if table == 'country':
            df = df.rename(columns={'COUNTRY': 'COUNTRY_CODE', 'LANGUAGE': 'LANGUAGE_CODE'})

        # Exporteer naar CSV
        df.to_csv(f"{sqlite_file}_{table}.csv", index=False, encoding='utf-8')

    sqlite_conn.close()

Verwerken van go_crm_train.sqlite...
  Exporteer tabel age_group naar CSV...
  Exporteer tabel country naar CSV...
  Exporteer tabel retailer_segment naar CSV...
  Exporteer tabel sales_territory naar CSV...
  Exporteer tabel retailer naar CSV...
  Exporteer tabel retailer_site naar CSV...
  Exporteer tabel retailer_type naar CSV...
  Exporteer tabel sales_demographic naar CSV...
  Exporteer tabel retailer_headquarters naar CSV...
  Exporteer tabel retailer_contact naar CSV...
Verwerken van go_sales_train.sqlite...
  Exporteer tabel country naar CSV...
  Exporteer tabel order_details naar CSV...
  Exporteer tabel order_header naar CSV...
  Exporteer tabel order_method naar CSV...
  Exporteer tabel product_line naar CSV...
  Exporteer tabel product_type naar CSV...
  Exporteer tabel product naar CSV...
  Exporteer tabel return_reason naar CSV...
  Exporteer tabel returned_item naar CSV...
  Exporteer tabel sales_branch naar CSV...
  Exporteer tabel sales_staff naar CSV...
  Exporteer ta

stap1

In [66]:
def get_sql_server_columns(table_name, cursor):
    cursor.execute(f"""
        SELECT COLUMN_NAME
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_NAME = '{table_name}'
    """)
    return [row.COLUMN_NAME for row in cursor.fetchall()]

stap2

In [67]:
def add_missing_columns(df, table_name, cursor, conn):
    # Haal bestaande kolommen op
    existing_columns = get_sql_server_columns(table_name, cursor)

    # Bepaal ontbrekende kolommen
    new_columns = [col for col in df.columns if col not in existing_columns]

    # Voeg nieuwe kolommen toe aan de tabel
    for col in new_columns:
        dtype = 'VARCHAR(MAX)'  # Default type, pas aan indien nodig
        cursor.execute(f"ALTER TABLE {table_name} ADD {col} {dtype}")
        conn.commit()
        print(f"Kolom {col} toegevoegd aan {table_name}.")

stap3

In [68]:
def insert_or_update_data(df, table_name, cursor, conn, primary_key='id'):
    # Stap 1: Maak een tijdelijke tabel aan
    temp_table = f"#{table_name}_temp"
    cursor.execute(f"""
        SELECT TOP 0 *
        INTO {temp_table}
        FROM {table_name}
    """)

    # Stap 2: Importeer data naar tijdelijke tabel
    df.to_sql(temp_table, conn, if_exists='append', index=False)

    # Stap 3: Merge data naar hoofdtafel (update bij conflict)
    columns = df.columns.tolist()
    update_columns = [f"target.{col} = source.{col}" for col in columns if col != primary_key]

    merge_sql = f"""
        MERGE INTO {table_name} AS target
        USING {temp_table} AS source
        ON target.{primary_key} = source.{primary_key}
        WHEN MATCHED THEN
            UPDATE SET {', '.join(update_columns)}
        WHEN NOT MATCHED THEN
            INSERT ({', '.join(columns)})
            VALUES ({', '.join([f'source.{col}' for col in columns])});
    """

    cursor.execute(merge_sql)
    conn.commit()
    print(f"Data voor {table_name} verwerkt (updates: {cursor.rowcount} rijen).")

csv naar database

In [79]:
import os
import pandas as pd
from sqlalchemy import create_engine, inspect, text
from sqlalchemy.engine import URL

# Stap 1: Maak een SQLAlchemy-engine
connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=XPS15\\SQLEXPRESS;"
    "DATABASE=gecombineerde_database2;"
    "Trusted_Connection=yes;"
)
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

# Stap 2: Functie om ontbrekende kolommen toe te voegen
def add_missing_columns(df, table_name, engine):
    inspector = inspect(engine)
    existing_columns = [col['name'] for col in inspector.get_columns(table_name)]

    for col in df.columns:
        if col not in existing_columns:
            dtype = 'VARCHAR(MAX)'  # Pas dit aan op basis van de data
            with engine.begin() as conn:
                conn.execute(text(f"ALTER TABLE {table_name} ADD {col} {dtype}"))
            print(f"Kolom {col} toegevoegd aan {table_name}.")

# Stap 3: Verwerk alle CSV-bestanden
csv_folder = "tabellen_in_csv_bestand/crm"
for csv_file in os.listdir(csv_folder):
    if csv_file.endswith(".csv"):
        table_name = os.path.splitext(csv_file)[0]  # Pas aan indien nodig
        csv_path = os.path.join(csv_folder, csv_file)

        print(f"Verwerken van {csv_file} voor tabel {table_name}...")
        df = pd.read_csv(csv_path)

        # Controleer of de tabel bestaat met inspect()
        inspector = inspect(engine)
        if not inspector.has_table(table_name):
            # Maak de tabel aan
            df.head(0).to_sql(table_name, engine, index=False)
            print(f"Tabel {table_name} aangemaakt.")

        # Voeg ontbrekende kolommen toe
        add_missing_columns(df, table_name, engine)

        # Importeer data (gebruik merge_data uit vorige voorbeeld)
        try:
            merge_data(df, table_name, primary_key='country_code', engine=engine)
        except Exception as e:
            print(f"Fout bij {table_name}: {e}")

Verwerken van go_crm_train.sqlite_age_group.csv voor tabel go_crm_train.sqlite_age_group...
Tabel go_crm_train.sqlite_age_group aangemaakt.
Fout bij go_crm_train.sqlite_age_group: (pyodbc.ProgrammingError) ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name '#go_crm_train.sqlite_age_group_temp'. (208) (SQLExecDirectW)")
[SQL: 
            MERGE INTO go_crm_train.sqlite_age_group AS target
            USING #go_crm_train.sqlite_age_group_temp AS source
            ON target.country_code = source.country_code
            WHEN MATCHED THEN
                UPDATE SET target.AGE_GROUP_CODE = source.AGE_GROUP_CODE, target.UPPER_AGE = source.UPPER_AGE, target.LOWER_AGE = source.LOWER_AGE
            WHEN NOT MATCHED THEN
                INSERT (AGE_GROUP_CODE, UPPER_AGE, LOWER_AGE)
                VALUES (source.AGE_GROUP_CODE, source.UPPER_AGE, source.LOWER_AGE);
        ]
(Background on this error at: https://sqlalche.me/e/20/f405)
Verwerken van go_c